# Simplified OpenMDAO Implementation

Here we will implement MDAO just to optimize the solar panel area.

First we need to import OpenMDAO

In [354]:
import math
import openmdao.api as om

Now we can make a `PEARL` class that models the PEARL servicing system for the optimization

# OpenMDAO Implementation
First we need to import OpenMDAO

In [355]:
class PEARLsimple(om.ExplicitComponent):
    """Simple PEARL model just to optimize solar panel size"""
    def setup(self):
        
        #Inputs
        self.add_input('P_hotel', 50.0, units="W", desc="nominal power draw from all sensors onboard the platform to keep it active")
        self.add_input('eta_s', 0.27, desc="solar cell efficiency")
        self.add_input('phi_s', 800.0, units="W/m**2", desc="incident solar irradiation")
        self.add_input('theta_bar', 55.0*math.pi/180, desc="average solar angle from the vertical over a day")
        self.add_input('I_d', 0.9, desc="inherent degradation")
        self.add_input('D', 0.005, desc="solar cell degradation per year?, depends on solar cell material")
        self.add_input('L_s', 10.0, desc="lifetime for solar panels in years")
        self.add_input('t_r', 12.0, units="h", desc="time the platform is recharging")
        self.add_input('E_AUV', 1900.0, units="W", desc="AUV battery capacity")
        self.add_input('t_service', 12.0, units="h", desc="duration of the time that AUV is connected to platform recharging and offloating data")
        self.add_input('gamma', 2.0, desc="AUVs serviced per day")
        self.add_input('A_s', 10.0, units="m**2", desc="solar panel area")
        self.add_input('t_mission', 24.0, units="h", desc="mission length")
        
        # Outputs
        self.add_output('P_recharge', 0.0, units="W", desc="energy harnessed recharging the platform")
        self.add_output('P_drawrecharge', 0.0, units="W", desc="power draw when PEARL is recharging")
        self.add_output('E_recharge', 0.0, units="W*h", desc="total energy collected to recharge teh platform's batteries")
        self.add_output('P_service', 0.0, units="W", desc="power draw to recharge the AUV")
        self.add_output('P_drawservice', 0.0, units="W", desc="power draw on the platform when the AUV is being serviced")
        self.add_output('min_diff', 0.0, desc="difference: E_recharge - E_hotel - E_service = 0 ")
        
        # Derivatives
        self.declare_partials('P_recharge', ['eta_s', 'phi_s', 'theta_bar', 'I_d', 'D', 'L_s', 'A_s'])
        self.declare_partials('P_service', ['E_AUV', 't_service','gamma'])
        self.declare_partials('E_recharge', ['P_recharge', 't_r'])
        self.declare_partials('P_drawrecharge',['P_hotel', 'eta_s', 'phi_s', 'theta_bar', 'I_d', 'D', 'L_s', 'A_s'])
        self.declare_partials('P_drawservice',['P_hotel','E_AUV', 't_service','gamma'])
        self.declare_partials('min_diff',['P_hotel', 'eta_s', 'phi_s', 'theta_bar', 'I_d', 'D', 'L_s', 'A_s','E_AUV', 't_service','gamma', 't_r','t_mission'])
   
    def compute(self, inputs, outputs):
        P_hotel = inputs['P_hotel']
        eta_s = inputs['eta_s']
        phi_s = inputs['phi_s']
        theta_bar = inputs['theta_bar']
        I_d = inputs['I_d']
        D = inputs['D']
        L_s = inputs['L_s']
        t_r = inputs['t_r']
        E_AUV = inputs['E_AUV']
        t_service = inputs['t_service']
        gamma = inputs['gamma']
        A_s = inputs['A_s']
        t_mission = inputs['t_mission']
        
        outputs['P_recharge'] = P_recharge = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
        outputs['P_drawrecharge'] = P_drawrecharge = P_hotel - P_recharge
        outputs['E_recharge'] = E_recharge = P_recharge * t_r
        outputs['P_service'] = P_service = E_AUV / t_service * gamma
        outputs['P_drawservice'] = P_drawservice = P_hotel - P_service
        outputs['min_diff'] = min_diff = E_recharge - P_hotel * t_mission - P_service * t_service

    def compute_partial(self, inputs, outputs, J): 
        P_hotel = inputs['P_hotel']
        eta_s = inputs['eta_s']
        phi_s = inputs['phi_s']
        theta_bar = inputs['theta_bar']
        I_d = inputs['I_d']
        D = inputs['D']
        L_s = inputs['L_s']
        t_r = inputs['t_r']
        E_AUV = inputs['E_AUV']
        t_service = inputs['t_service']
        gamma = inputs['gamma']
        A_s = inputs['A_s']
        t_mission = inputs['t_mission']
        
        J['P_recharge','eta_s'] = phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
        J['P_recharge','phi_s'] = eta_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
        J['P_recharge','theta_bar'] = -eta_s * phi_s * math.sin(theta_bar) * I_d * (1-D)**L_s * A_s
        J['P_recharge','I_d'] = eta_s * phi_s * math.cos(theta_bar) * (1-D)**L_s * A_s
        J['P_recharge','D'] = -eta_s * phi_s * math.cos(theta_bar) * I_d * L_s * (1-D)**(L_s-1) * A_s
        J['P_recharge','L_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s *log(1-D) * A_s
        J['P_recharge','A_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s
        
        J['P_drawrecharge','P_hotel'] = 1.0
        J['P_drawrecharge','eta_s'] = - phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
        J['P_drawrecharge','phi_s'] = - eta_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
        J['P_drawrecharge','theta_bar'] = eta_s * phi_s * math.sin(theta_bar) * I_d * (1-D)**L_s * A_s
        J['P_drawrecharge','I_d'] = - eta_s * phi_s * math.cos(theta_bar) * (1-D)**L_s * A_s
        J['P_drawrecharge','D'] = eta_s * phi_s * math.cos(theta_bar) * I_d * L_s * (1-D)**(L_s-1) * A_s
        J['P_drawrecharge','L_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s *log(1-D) * A_s
        J['P_drawrecharge','A_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s
        
        J['E_recharge', 't_r'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s 
        J['E_recharge','eta_s'] = phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
        J['E_recharge','phi_s'] = eta_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
        J['E_recharge','theta_bar'] = -eta_s * phi_s * math.sin(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
        J['E_recharge','I_d'] = eta_s * phi_s * math.cos(theta_bar) * (1-D)**L_s * A_s * t_r
        J['E_recharge','D'] = -eta_s * phi_s * math.cos(theta_bar) * I_d * L_s * (1-D)**(L_s-1) * A_s * t_r
        J['E_recharge','L_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s *log(1-D) * A_s * t_r
        J['E_recharge','A_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * t_r
        
        J['P_service', 'E_AUV'] = 1.0 / t_service * gamma
        J['P_service', 't_service'] = - (E_AUV / t_service ** 2) * gamma
        J['P_service', 'gamma'] = E_AUV / t_service
        
        J['P_drawservice', 'P_hotel'] = - E_AUV / t_service * gamma
        J['P_drawservice', 'E_AUV'] = - 1.0 / t_service * gamma
        J['P_drawservice', 't_service'] = (E_AUV / t_service ** 2) * gamma
        J['P_drawservice', 'gamma'] = - E_AUV / t_service
        
        J['min_diff', 't_r'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s 
        J['min_diff','eta_s'] = phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
        J['min_diff','phi_s'] = eta_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
        J['min_diff','theta_bar'] = -eta_s * phi_s * math.sin(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
        J['min_diff','I_d'] = eta_s * phi_s * math.cos(theta_bar) * (1-D)**L_s * A_s * t_r
        J['min_diff','D'] = -eta_s * phi_s * math.cos(theta_bar) * I_d * L_s * (1-D)**(L_s-1) * A_s * t_r
        J['min_diff','L_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s *log(1-D) * A_s * t_r
        J['min_diff','A_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * t_r
        J['min_diff','P_hotel'] = - t_mission
        J['min_diff','t_mission'] = - P_hotel
        J['min_diff','E_AUV'] = - gamma
        J['min_diff','gamma'] = - E_AUV

The class declaration, `class PEARLsimple(om.ExplicitComponent):` shows that our class, `PEARLsimple` inherits off of the `ExplicitComponent` class in OpenMDAO.  Other types of components are described in the OpenMDAO docs [here](http://openmdao.org/twodocs/versions/latest/_srcdocs/packages/openmdao.components.html).

The `ExplicitComponent` class provides a template for the user to: 
- Declare their input and output variables in the `setup` method
- Calculate the outputs from the inputs in the `compute` method.  In an optimization loop, this is called at every iteration.
- Calculate analytical gradients of outputs with respect to inputs in the `compute_partials` method.

The variable declarations take the form of `self.add_input` or `self.add_output` where a variable name and default/initial vaue is assigned.  The value declaration also tells the OpenMDAO internals about the size and shape for any vector or multi-dimensional variables.  Other optional keywords that can help with code documentation and model consistency are `units=` and `desc=`.

### Working with analytical derivatives derivatives

We need to tell OpenMDAO which derivatives will need to be computed. That happens in the following lines:

```python
self.declare_partials('P_recharge', ['eta_s', 'phi_s', 'theta_bar', 'I_d', 'D', 'L_s', 'A_s'])
self.declare_partials('P_service', ['E_AUV', 't_service','gamma'])
self.declare_partials('E_recharge', ['P_recharge', 't_r'])
self.declare_partials('P_drawrecharge',['P_hotel', 'eta_s', 'phi_s', 'theta_bar', 'I_d', 'D', 'L_s', 'A_s'])
self.declare_partials('P_drawservice',['P_hotel','E_AUV', 't_service','gamma'])
self.declare_partials('min_diff',['P_hotel', 'eta_s', 'phi_s', 'theta_bar', 'I_d', 'D', 'L_s', 'A_s','E_AUV', 't_service','gamma', 't_r','t_mission'])```

Note that lines like `self.declare_partials('P_service', ['E_AUV', 't_service','gamma'])` references the derivatives \\(\partial P_{service} / \partial E_{AUV}\\), \\(\partial P_{service} / \partial t_{service}\\) and \\(\partial P_{service} / \partial \gamma\\).

The Jacobian in which we provide solutions to the derivatives is

```python
J['P_recharge','eta_s'] = phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
J['P_recharge','phi_s'] = eta_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
J['P_recharge','theta_bar'] = -eta_s * phi_s * math.sin(theta_bar) * I_d * (1-D)**L_s * A_s
J['P_recharge','I_d'] = eta_s * phi_s * math.cos(theta_bar) * (1-D)**L_s * A_s
J['P_recharge','D'] = -eta_s * phi_s * math.cos(theta_bar) * I_d * L_s * (1-D)**(L_s-1) * A_s
J['P_recharge','L_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s *log(1-D) * A_s
J['P_recharge','A_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s
        
J['P_drawrecharge','P_hotel'] = 1.0
J['P_drawrecharge','eta_s'] = - phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
J['P_drawrecharge','phi_s'] = - eta_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s
J['P_drawrecharge','theta_bar'] = eta_s * phi_s * math.sin(theta_bar) * I_d * (1-D)**L_s * A_s
J['P_drawrecharge','I_d'] = - eta_s * phi_s * math.cos(theta_bar) * (1-D)**L_s * A_s
J['P_drawrecharge','D'] = eta_s * phi_s * math.cos(theta_bar) * I_d * L_s * (1-D)**(L_s-1) * A_s
J['P_drawrecharge','L_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s *log(1-D) * A_s
J['P_drawrecharge','A_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s
        
J['E_recharge', 't_r'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s 
J['E_recharge','eta_s'] = phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
J['E_recharge','phi_s'] = eta_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
J['E_recharge','theta_bar'] = -eta_s * phi_s * math.sin(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
J['E_recharge','I_d'] = eta_s * phi_s * math.cos(theta_bar) * (1-D)**L_s * A_s * t_r
J['E_recharge','D'] = -eta_s * phi_s * math.cos(theta_bar) * I_d * L_s * (1-D)**(L_s-1) * A_s * t_r
J['E_recharge','L_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s *log(1-D) * A_s * t_r
J['E_recharge','A_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * t_r
        
J['P_service', 'E_AUV'] = 1.0 / t_service * gamma
J['P_service', 't_service'] = - (E_AUV / t_service ** 2) * gamma
J['P_service', 'gamma'] = E_AUV / t_service
        
J['P_drawservice', 'P_hotel'] = - E_AUV / t_service * gamma
J['P_drawservice', 'E_AUV'] = - 1.0 / t_service * gamma
J['P_drawservice', 't_service'] = (E_AUV / t_service ** 2) * gamma
J['P_drawservice', 'gamma'] = - E_AUV / t_service

J['min_diff', 't_r'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s 
J['min_diff','eta_s'] = phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
J['min_diff','phi_s'] = eta_s * math.cos(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
J['min_diff','theta_bar'] = -eta_s * phi_s * math.sin(theta_bar) * I_d * (1-D)**L_s * A_s * t_r
J['min_diff','I_d'] = eta_s * phi_s * math.cos(theta_bar) * (1-D)**L_s * A_s * t_r
J['min_diff','D'] = -eta_s * phi_s * math.cos(theta_bar) * I_d * L_s * (1-D)**(L_s-1) * A_s * t_r
J['min_diff','L_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s *log(1-D) * A_s * t_r
J['min_diff','A_s'] = eta_s * phi_s * math.cos(theta_bar) * I_d * (1-D)**L_s * t_r
J['min_diff','P_hotel'] = - t_mission
J['min_diff','t_mission'] = - P_hotel
J['min_diff','E_AUV'] = - gamma
J['min_diff','gamma'] = - E_AUV```

In OpenMDAO, multiple components can be connected together inside of a Group.  There will be some other new elements to review, so let's take a look:

### Recharge Group:

In [356]:
class Recharge(om.Group):
    """
    Group containing the charging equations for determinig PEARL solar size based on recharging needs.
    """

    def setup(self):
        indeps = self.add_subsystem('indeps', om.IndepVarComp(), promotes=['*'])
        indeps.add_output('P_hotel', 50.0, units="W")
        indeps.add_output('eta_s', 0.27)
        indeps.add_output('phi_s', 800.0, units="W/m**2")
        indeps.add_output('theta_bar', 55.0*math.pi/180)
        indeps.add_output('I_d', 0.9)
        indeps.add_output('D', 0.005)
        indeps.add_output('L_s', 10.0)
        indeps.add_output('t_r', 12.0, units="h")
        indeps.add_output('E_AUV', 1900.0, units="W")
        indeps.add_output('t_service', 12.0, units="h")
        indeps.add_output('gamma', 2.0)
        indeps.add_output('A_s', 1.0, units="m**2")
        indeps.add_output('t_mission', 24.0, units="h")
        
        self.add_subsystem('solar_panels', PEARLsimple(), promotes=['*'])

The `Recharge` class derives off of the OpenMDAO `Group` class, which is typically the top-level class that is used in an analysis.  The OpenMDAO `Group` class allows you to cluster models in hierarchies. We can put multiple components in groups. We can also put other groups in groups.

Components are added to groups with the `self.add_subsystem` command, which has two primary arguments.  The first is the string name to call the subsystem that is added and the second is the component or sub-group class instance.  A common optional argument is `promotes=`, which elevatest the input/ouput variable string names to the top-level namespace.  The `Recharge` group shows examples where the `promotes=` can be passed a list of variable string names or the `'*'` wildcard to mean all input/output variables.

The first subsystem that is added is an `IndepVarComp`, which are the independent variables of the problem.  Subsystem inputs that are not tied to other subsystem outputs should be connected to an independent variables.  For optimization problems, design variables must be part of an `IndepVarComp`.  In the PEARL recharging problem, we have `P_hotel`, `eta_s`, `phi_s`,`theta_bar`,`I_d`,`D`,`L_s`, `t_r`, `E_AUV`, `t_service`, `gamma`, and `A_s`.  Note that they are promoted to the top level namespace, otherwise we would have to access them by `'indeps.x'` and `'indeps.z'`.

The next subsystem that is added is an instance of the component we created above:
```python
self.add_subsystem('solar_panels', PEARLsimple(), promotes=['*'])
```

The `promotes=` can also serve to connect variables.  In OpenMDAO, two variables with the same string name in the same namespace are automatically connected.  By promoting the same variable string names as in the `IndepCarComp`, they are automatically connected.  For variables that are not connected in this way, explicit connect statements are required, which is demonstrated in the next tutorial.

## Let's optimize our system!

Even though we have all the pieces in a `Group`, we still need to put them into a `Problem` to be executed.  The `Problem` instance is where we can assign design variables, objective functions, and constraints.  It is also how the user interacts with the `Group` to set initial conditions and interrogate output values.

First, we instantiate the `Problem` and assign an instance of `Recharge` to be the root model:

In [357]:
prob = om.Problem()
prob.model = Recharge()

Next we assign an optimization driver to the problem instance. If we only wanted to evaluate the model once and not optimize, then a driver is not needed:

In [358]:
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'

With the optimization driver in place, we can assign design variables, objective(s), and constraints. Any IndepVarComp can be a design variable and any model output can be an objective or constraint.

We want to minimize the objective, which is in line with what OpenMDAO will want to do as it is consistent with the standard optimization problem statement. 

In [359]:
prob.model.add_design_var('gamma', lower=0.0, upper=10.0)
prob.model.add_design_var('A_s', lower=0.0, upper=100.0)
prob.model.add_objective('min_diff', scaler=1.0)

Now we can run the optimization:

In [360]:
prob.setup()
prob.run_driver()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -3727.377446835061
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1
Optimization Complete
-----------------------------------


False

## Finally, the result:
Above, we see a summary of the steps in our optimization. Don't worry about the output `False` for now. Next, we print out the values we care about and list all of the inputs and outputs that are problem used. 

In [361]:
print('Energy to recharge = ', prob['E_recharge'])
print('min_diff = ', prob['min_diff'])
print('Solar panel area =', prob['A_s'], 'm^2')
print('Number of AUVs serviced per day =', prob['gamma'])
all_inputs  = prob.model.list_inputs(values=True)
all_outputs = prob.model.list_outputs(values=True)

Energy to recharge =  [1272.62255316]
min_diff =  [-3727.37744684]
Solar panel area = [1.] m^2
Number of AUVs serviced per day = [2.]
13 Input(s) in 'model'
----------------------

varname         value       
--------------  ------------
top
  solar_panels
    P_hotel     [50.]       
    eta_s       [0.27]      
    phi_s       [800.]      
    theta_bar   [0.95993109]
    I_d         [0.9]       
    D           [0.005]     
    L_s         [10.]       
    t_r         [12.]       
    E_AUV       [1900.]     
    t_service   [12.]       
    gamma       [2.]        
    A_s         [1.]        
    t_mission   [24.]       


19 Explicit Output(s) in 'model'
--------------------------------

varname             value           
------------------  ----------------
top
  indeps
    P_hotel         [50.]           
    eta_s           [0.27]          
    phi_s           [800.]          
    theta_bar       [0.95993109]    
    I_d             [0.9]           
    D               [0.0

## Visualizing the results
Let's visualize the model using an N2 diagram

In [362]:
#from openmdao.api import n2
#n2(prob)

In [363]:
# import openmdao.api as om

Now we can make a `PEARL` class that models the PEARL servicing system for the optimization

In [364]:
# class PEARL(om.ExplicitComponent):
#     """Simple PEARL model based on governingEqns.ipynb"""
#     def setup(self):
        
#         # Inputs
#         self.add_input('P_hotel', 50.0, units="W", desc="nominal power draw from all sensors onboard the platform to keep it active")
#         self.add_input('eta_s', 0.27, desc="solar cell efficiency")
#         self.add_input('phi_s', 800.0, units="W/m**2", desc="incident solar irradiation")
#         self.add_input('theta_bar', 55.055.0*180.0/math.pi, desc="average solar angle from the vertical over a day")
#         self.add_input('I_d', 0.9, desc="inherent degradation")
#         self.add_input('D', 0.005, desc="solar cell degradation, depends on solar cell material")
#         self.add_input('L_s', 10.0, units="years", desc="lifetime for solar panels")
#         self.add_input('t_r', 12.0, units="hr", desc="time the platform is recharging")
#         self.add_input('E_AUV', 1900.0, units="W", desc="AUV battery capacity")
#         self.add_input('t_service', 12.0, units="hours", desc="duration of the time that AUV is connected to platform recharging and offloating data")
#         self.add_input('gamma', 0.2, desc="AUVs serviced per day")
#         self.add_input('P_comms', 20.0, units="20", desc="power required to transmit data to the satellite")
#         self.add_input('t_mission', 24.0, units="hr", desc="total duration of the mission of the platform")
#         self.add_input('t_c', 4.0, units="hr", desc="total time platform is transmitting data per mission")
#         self.add_input('t_m', 1.2, units="hr", desc="total time the platform is moving per mission")
#         self.add_input('rho', 1023, units="kg/m**3", desc="density of seawater")
#         self.add_input('C_d', 1.0, desc="drag coefficient")
#         self.add_input('eta_m', 0.75, desc="propulsion efficiency")
#         self.add_input('V', 1.0, units="m/s", desc="velocity of platform")
#         self.add_input('m_comms', 50.0, units="kg", desc="mass of the comms system")
#         self.add_input('m_propulsion', 50.0, units="kg", desc="mass of the propulsion system on the platform")
#         self.add_input('mu_battery', 200.0, units="W*hr/kg", desc="battery specific energy density for Li-ion")
#         self.add_input('DOD', 0.70, desc="depth of discharge of battery")
#         self.add_input('eta_battery', 0.85, desc="transmission efficiency between the battery and the load")
#         self.add_input('N', 1, desc="number of batteries")
#         self.add_input('nu_battery', 215.0, units="W*hr/L volume", desc="battery volumetric energy density")
#         self.add_input('h_f', 2.0, units="m", desc="top float of platform distance to SWL")
#         self.add_input('t_f', 4.0, units="m", desc="thickness of top float")
#         self.add_input('t_s', 6.0, units="m", desc="length of middle spar support column")
#         self.add_input('D_s', 3.0, units="m", desc="diameter of middle spar support column")
#         self.add_input('t_d', 2.0, units="m", desc="thickness of bottom dampening plate")
#         self.add_input('D_d', 4.0, units="m", desc="diametet of bottom dampening plate")
        
#         # Outputs
#         self.add_output('P_recharge', 0.0, units="W", desc="energy harnessed recharging the platform")
#         self.add_output('P_draw,recharge', 0.0, units="W", desc="power draw when PEARL is recharging")
#         self.add_output('A_s', 0.0, units="m**2", desc="solar panel area")
#         self.add_output('E_recharge', 0.0, units="W*hr", desc="total energy collected to recharge teh platform's batteries")
#         self.add_output('P_service', 0.0, units="W", desc="power draw to recharge the AUV")
#         self.add_output('P_draw,service', 0.0, units="W", desc="power draw on the platform when the AUV is being serviced")
#         self.add_output('P_draw,comms', 0.0, units="W", desc="power draw of PEARL when it is transmitting data")
#         self.add_output('P_move', 0.0, units="W", desc="power required to move the platform")
#         self.add_output('P_draw,move', 0.0, units="W", desc="power draw of the platform when it's moving")
#         self.add_output('S_w', 0.0, units="m**2", desc="wetted surface area of the platform")
#         self.add_output('m_platform', 0.0, units="kg", desc="mass of the platform")
#         self.add_output('m_batteries', 0.0, units="kg", desc="mass of the batteries on the platform")
#         self.add_output('m_solar', 0.0, units="kg", desc="mass of the solar panels on the platform")
#         self.add_output('m_structure', 0.0, units="kg", desc="mass of the structure of the platform")
#         self.add_output('E_battery', 0.0, units="W*hr", desc="power storage capacity of the batteries")
#         self.add_output('E_req', 0.0, units="W*hr", desc="total power required by PEARL for mission")
#         self.add_output('V_battery', 0.0, units="m**3", desc="volume of the battery")
#         self.add_output('D_f', 0.0, units="m", desc="diameter of top float")
#         self.add_output('z_B', 0.0, units="m", desc="distance to center of bouyancy (measured from SWL)")
#         self.add_output('z_G', 0.0, units="m", desc="distance to center of gravity (measured from SWL)")
#         self.add_output('F_B', 0.0, unis="N", desc="bouyant force")
        
#         # Derivatives
#         self.declare_partials('P_recharge', ['eta_s', 'phi_s', 'theta_bar', 'I_d', 'D', 'L_s', 'A_s'])
#         self.declare_partials('P_service', ['E_AUV', 't_service','gamma'])
#         self.declare_partials('P_move',['rho','C_D','V','eta_m','S_w'])
#         self.declare_partials('S_w',['D_f','t_f','D_s','t_s','D_d','t_d'])         
        
    